# Day 5

Tets input:

```
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
```

You basically need to create a series of mapping functions to go all the way from seed number to location. Report the lowest location. In the real input, numbers get huge.

In [1]:
test_input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [2]:
"".isspace()

False

In [3]:
# create a "mapper" class. This will be easier than using a dict. A mapper will take inputs from the statement and output numbers correctly.

class SeedMap:
    def __init__(self, name, container):
        self.name = name
        container = [[int(e) for e in line.split(" ")] for line in container]
        self.container = container

    def translate(self,num):
        for row in self.container:
            if num >= row[1] and num < row[1] + row[2]:
                return row[0] + (num - row[1])
        return num

def make_seed_maps(instructions_list):
    map_holder = []
    current_name = ""
    current_container = []
    for l in instructions_list[2:]:
        if not l:
            map_holder.append(SeedMap(current_name,current_container))
        elif l[0].isalpha():
            current_name = l
            current_container = []
        elif l[0].isnumeric():
            current_container.append(l)
    map_holder.append(SeedMap(current_name,current_container))
    return map_holder


def part1(instructions):
    # get seed nums
    # foor each group of instructions, make a mapper function
    instructions_list = instructions.split("\n")
    seeds = [int(s) for s in instructions_list[0].split(":")[1].strip().split(" ")]
    seed_maps_list = make_seed_maps(instructions_list)
    final_outputs = []
    for seed in seeds:
        s_num = seed
        for m in seed_maps_list:
            s_num = m.translate(s_num)
        final_outputs.append(s_num)
    return min(final_outputs)



In [4]:
part1(test_input) # translations are 82, 43, 86, 35 - 35 is answer 

35

In [5]:
with open("day5input.txt","r") as f:
    real_input = f.read()

In [6]:
part1(real_input)

457535844

## Part 2
same, but the seed number line is actually a list of two-ples. THe first is a start number, the second is a range, inclusive. still report the lowest num.

hmm... the second numbers are in the scale of millions (up to 100M). Might not be able to brute force this one. Gotta think a bit... ok. gotta make a "range" 

In [50]:
class SeedRange:
    def __init__(self, start, length):
        self.start = start
        self.length = length
        self.end = start+length-1

    def __str__(self):
        return f"{self.start}  {self.length}"

    def __repr__(self):
        return self.__str__()

class SeedMap:
    def __init__(self, name, container):
        self.name = name
        container = [[int(e) for e in line.split(" ")] for line in container] 
        self.container = container #0 is destination, #1 is  source, 2 is range

    def translate_single(self,num):
        for row in self.container:
            if num >= row[1] and num < row[1] + row[2]:
                return row[0] + (num - row[1])
        return num

    def translate_range(self, sr: SeedRange) -> list[SeedRange]:
        seed_range_holder = [sr,]
        for row in self.container:
            new_holder = []
            for srange in seed_range_holder:
                
                if srange.end < row[1]:
                    new_holder.append(srange)
                elif srange.start > row[1]+row[2] - 1:
                    new_holder.append(srange)
                
                elif srange.start < row[1] and srange.end < row[1] + row[2]:
                    new_holder.append(SeedRange(srange.start,row[1]-srange.start)) #ok
                    new_holder.append(SeedRange(row[0],srange.end - row[1] + 1))
                    
                elif srange.start < row[1] and srange.end >= row[1] + row[2]:
                    new_holder.append(SeedRange(srange.start,row[1]-srange.start))
                    new_holder.append(SeedRange(row[0],row[2]))
                    new_holder.append(SeedRange(row[1]+row[2], srange.end - (row[1]+row[2]) + 1))
                
                elif srange.start < row[1]+row[2] and srange.end >= row[1] + row[2]:
                    new_holder.append(SeedRange(row[0]+(srange.start-row[1]),row[1]+row[2] - srange.start))
                    new_holder.append(SeedRange(row[1]+row[2], srange.end - (row[1]+row[2])))
                
                elif srange.start > row[1] and srange.start+srange.length <= row[1]+row[2]:
                    new_holder.append(SeedRange(row[0]+(srange.start-row[1]),srange.length))
            
            seed_range_holder = new_holder
            # print(new_holder)
        return seed_range_holder


def fix_seeds(seeds):
    all_seeds = []
    for i in range(0,len(seeds),2):
        s1 = seeds[i]
        s2 = seeds[i+1]
        all_seeds.append(SeedRange(s1,s2))
    return all_seeds

def part2(instructions):
    # get seed nums
    # foor each group of instructions, make a mapper function
    instructions_list = instructions.split("\n")
    seeds = [int(s) for s in instructions_list[0].split(":")[1].strip().split(" ")]
    seeds = fix_seeds(seeds)
    seed_maps_list = make_seed_maps(instructions_list)
    final_outputs = []
    for seed in seeds:
        seed_container = [seed,]
        for m in seed_maps_list:
            new_container = []
            for s in seed_container:
                new_container += m.translate_range(s)
            seed_container = new_container
        final_outputs += [s.start for s in seed_container]
    return min(final_outputs)

In [53]:
part2(test_input) #46

57

In [52]:
part2(real_input)

11992204

In [ ]:
# 3179822 too low
# 11992204 to low